🇧🇷 CPC-CONSTITUTE
---

## Centro de Política Parada: Constitute

<img src="img/cpc.jpg" width="50%" heigth="50%" />

Este projeto visa a criação de uma plataforma que disponibilize **informações constitucionais sobre o funcionamento dos sistemas de governo** em democracias a nível global, por via da interação com a API do **[Project Constitute](https://www.constituteproject.org/)**, além de redes de notícias, enciclopédias e citações acadêmicas.

O **[Centro de Política Comparada (CPC)](http://www.cpc.ufes.br)** é um núcleo de pesquisas vinculado ao Departamento de Ciências Sociais (DCSO) da Universidade Federal do Espírito Santo (Ufes), fundado e registrado no Diretório dos Grupos de Pesquisa no Brasil, do CNPq, em 2016.

### Estágio de desenvolvimento

O projeto encontra-se com o código para obtenção de dados e interação com API já desenvolvido, podendo ser executado localmente ou por plataformas virtuais como o [Colab](https://colab.research.google.com/). Sugestões de futuras atualizações incluem a revisão e reestruturação dos arquivos do repositório, focado no desenvolvimento de um webaplicativo que vise integrar os resultados do código desenvolvido. Funções desenvolvidas:

* Interagir com a API do Constitute Project para realizar buscas e demais ações;
* Disponibilizar o acesso a dados sobre o modo de organização dos sistemas de governo;
* Comparar características entre sistemas presidencialistas, parlamentaristas e híbridos;
* Possibilitar pesquisas de excertos e temáticas presentes nos textos constitucionais;
* Buscar publicações associadas às constituições pesquisadas e a artigos acadêmicos;
* Capturar notícias recentes do conceito ou país pesquisado pelo MediaCloud/News API;
* Coletar dados de tweets publicados a priori na última semana com "política" atualmente;
* Pesquisar correlações entre páginas da Wikipédia sobre os termos de busca utilizados.

### Importar bibliotecas

In [ ]:
import sys
from os.path import dirname, realpath
path = realpath('.')
if path not in sys.path:
    sys.path.append(path)

In [ ]:
import networkx as nx
import pandas as pd
import plotly.offline as py
import webbrowser

import src as cpc
import cpc.__init__

py.init_notebook_mode(connected=True)

%load_ext autoreload
%autoreload 2

### Importar credenciais

Necessário para obter dados das plataformas `News API`, `MediaCloud` e `Twitter`.

In [ ]:
mcloud_key   = cpc.cfg.MCLOUD_KEY
newsapi_key  = cpc.cfg.NEWSAPI_KEY
app_keys     = cpc.cfg.TWITTER_KEYS
app_key      = cpc.cfg.TWITTER_TOKENS[0][0]
app_secret   = cpc.cfg.TWITTER_TOKENS[0][1]
oauth_token  = cpc.cfg.TWITTER_TOKENS[0][2]
oauth_secret = cpc.cfg.TWITTER_TOKENS[0][3]

#### Definir credenciais manualmente (opcional)

In [ ]:
#mcloud_key   = ""
#newsapi_key  = ""
#app_keys     = ""
#app_key      = ""
#app_secret   = ""
#oauth_token  = ""
#oauth_secret = c""

### Constitute

URL base: `https://www.constituteproject.org/service/<method_name><params>`

* Listas de objetos: `constitutions`, `topics`, `regions`

* Busca por tópicos: `constopicsearch`, `sectionstopicsearch`

* Busca textual: `textsearch`

* Requisitar constituição para web: `html`

#### Constituições

In [ ]:
c = constitute_query('constitutions', ''); C=c
# c = constitute_query('constitutions', '?region=Africa')                # <-- constituições da África
# c = constitute_query('constitutions', '?region=Africa&region=Europe')  # <-- constituições da África e Europa
# c = constitute_query('constitutions', '?country=Ukraine')              # <-- constituições da Ucrânia
# c = constitute_query('constitutions', '?from_year=1954&to_year=2000')  # <-- constituições de 1954 a 2000
# c = constitute_query('constitutions', '?region=Europe&from_year=1960') # <-- constituições europeias desde 1960
# c = constitute_query('constitutions', '?lang=en')                      # <-- constituições em língua inglesa
# c = constitute_query('constitutions', '?historic=false')               # <-- constituições em vigor

#### Tópicos

In [ ]:
c = constitute_query('topics', '?lang=en') # <-- tópicos disponíveis em um idioma específico

#### Regiões

In [ ]:
c = constitute_query('locations', '?lang=en') # <-- países ou regiões cujo idioma é o inglês

#### Busca por tópicos de uma constituição

In [ ]:
c = constitute_query('constopicsearch', '?key=amend&lang=en')
# c = constitute_query('constopicsearch', '?key=amend&region=Africa')
# c = constitute_query('constopicsearch', '?key=amend&region=Africa&region=Europe')
# c = constitute_query('constopicsearch', '?key=amend&country=Ukraine')
# c = constitute_query('constopicsearch', '?key=amend&from_year=1954&to_year=2000')
# c = constitute_query('constopicsearch', '?key=amend&historic=true')

#### Busca por seções de uma constituição

In [ ]:
c = constitute_query('sectionstopicsearch', '?key=amend&cons_id=Afghanistan_2004&lang=en')
# c = constitute_query('sectionstopicsearch', '?key=hogterm&cons_id=China_2004&lang=en')
# c = constitute_query('sectionstopicsearch', '?key=cabsel&cons_id=Australia_1985&lang=ar')

#### Busca textual

In [ ]:
c = constitute_query('textsearch', '?q=prime%20minister&lang=en')
# c = constitute_query('textsearch', '?q=president&cons_id=Afghanistan_2004')

#### Conteúdo em HTML

In [ ]:
c = constitute_query('html', '?cons_id=Australia_1985&lang=en')
# c = constitute_query('html', '?cons_id=Australia_1985&lang=ar')

#### Carregar num data frame

In [ ]:
df = concat_dict(C); df.head(1)

### Notícias

* Artigos mais vistos: `news_top_headlines`

* Todos os artigos: `news_get_everything`

In [ ]:
news = news_top_headlines('bolsonaro', lang='pt', days=7); print('Got', len(news), 'news')

In [ ]:
news = news_get_everything('bolsonaro', lang='pt', days=7); print('Got', len(news), 'news')

In [ ]:
# count = 1
# for a in news:
#     print('\n=> Notícia',count); count+=1
#     for item in a: print('* %s = %s' % (item,a[item]))

### Google

* Resultados por busca textual: `google_search`

* Abre uma página com o PDF retornado: `google_pdf`

In [ ]:
g = google_search('Constituição+Brasil+filetype:pdf')

In [ ]:
pdf = g[0].link; webbrowser.open(pdf); print(pdf)

In [ ]:
# print('Got', len(g), 'results')
# count = 1
# for a in g:
#     print('\n=> Página', count,
#     '\n* Name:', a.name,
#     '\n* Link:', a.link,
#     '\n* Description:', a.description,
#     '\n* Number_of_results:', a.number_of_results); count += 1

### Wikipedia

* Conteúdo de uma página: `wiki_page`
* Buscar páginas por nome:`wiki_search`
* Sumário de uma página: `wiki_summary`
* Apresentar páginas relacionadas:`wiki_graph`

In [ ]:
w = wiki_page("Democracia", lang='pt'); print(w.title, '\n', w.url, '\n\nDescrição:',  w.content)

In [ ]:
w = wiki_search("Democracia", "pt");w

In [ ]:
w = wiki_summary("Democracia", "pt");w

In [ ]:
w = wiki_graph(["Democracia","Governo","Política","Estado"], "pt", n=5)
iplot(scatter_plot(w,layout='forceatlas2',name='Artigos relacionados na Wikipedia'))

### Twitter

* Já publicados: `collect_twitter`
* Em tempo real: `stream_tweets`
* Viralizando: `trending_topics`

In [ ]:
collect_twitter('constituição+política', lang='pt', limit=10, app_keys=TWITTER_KEYS);tweets=pd.read_csv('tweets.csv')

In [ ]:
stream_tweets('trump', limit=10, app_key=app_key, app_secret=app_secret, post_url="",
              oauth_token=oauth_token, oauth_secret=oauth_secret);tweets=pd.read_csv('tweets.csv')

In [ ]:
trending_topics('BR', post_url=None, app_keys=TWITTER_KEYS)

### Referências

* [Comparative Constitutions Project](https://comparativeconstitutionsproject.org/)

* [Constitute API Documentation](https://docs.google.com/document/d/1wATS_IAcOpNZKzMrvO8SMmjCgOZfgH97gmPedVxpMfw/pub)

* [Constitute Project](https://www.constituteproject.org)

* [constviz @ GitHub](https://github.com/thequbit/constviz/blob/master/README.md)

* [Google Search API @ GitHub](https://github.com/abenassi/Google-Search-API)

* [Glossary of constitutional terms](https://www.infoplease.com/chemistry/glossary/glossary-constitutional-terms)

* [Scraping Constitute](https://sites.psu.edu/bdssblog/author/rbm166/)